In [6]:
import numpy as np
import sys
import torch

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
from vandy_taggers.modules.models.part import ParticleTransformer, PartTrainer
from vandy_taggers.utils.data_loader import DataLoader

In [9]:
BS = 1
model = ParticleTransformer(num_classes = 6,
                 num_enc = 8,
                 num_head = 8,
                 embed_dim = 128,
                 cpf_dim = 16,
                 npf_dim = 8,
                 vtx_dim = 14,
                 for_inference = False,)
model.to(device);
model.eval();

In [10]:
model = torch.load("../../saved_models/best_model.pt")

In [11]:
OUT_CONVERTED_PATH = "../../../Datalake/converted/*test*"
test_loader = DataLoader(OUT_CONVERTED_PATH, BS).get_loader()

In [12]:
all_predictions = []
all_targets = []
model.eval()
with torch.no_grad():
    for (X, y) in test_loader:
        predictions = model(X)
        all_predictions.append(predictions)
        all_targets.append(y)

In [13]:
predicted_classes = np.concatenate([torch.argmax(p, dim=1).cpu().numpy() for p in all_predictions])
ground_truth = np.concatenate([torch.argmax(t, dim=1).cpu().numpy() for t in all_targets])

In [14]:
acc = np.mean(predicted_classes == ground_truth)
acc

0.6540803897685749